# Import necessary package

In [1]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers
!pip install scipy
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
import tensorflow as tf
from transformers import RobertaTokenizer, RobertaModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 163 kB 47.1 MB/s 
     |████████████████████████████████| 7.6 MB 36.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open('PennsylvaniaMidterms11Aug11Oct2').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
import pandas as pd
df=pd.DataFrame.from_records(rows)

In [4]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

# data preprocessing

In [5]:
df['SentimentScore']= pd.to_numeric(df['SentimentScore'])
df['Subjectivity']= pd.to_numeric(df['Subjectivity'])
df['Sarcasm']= pd.to_numeric(df['Sarcasm'])
df['Faction']= pd.to_numeric(df['Faction'])
final_df=df.loc[df['SentimentScore'].isin([0,1,2])]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
import re
import nltk

def clean_content(content):
  content=re.sub('#','',content)
  content = re.sub(r'@[^(JohnFetterman)][^(DrOz)]\w+', '', content)
  content = re.sub('@', '', content)
  content = re.sub(r'\\u\w+', '', content)
  content = re.sub(r"http\S+", "", content)
  content = re.sub("\n", "", content)
  # content = re.sub(r'[^\w\s\.]', '', content)
  # content=' '.join(s for s in content.split() if not any(c.isdigit() for c in s))
  # content=content.lower()
  return content

final_df['cleaned_tweets']=final_df['Content'].apply(clean_content)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [7]:
sarcasm = final_df[final_df['Sarcasm'].isin([0,1])]

In [8]:
sarcasm

,Date,Content,,,,,,,,,,,,,SentimentScore,Subjectivity,Sarcasm,Faction,cleaned_tweets
1,2022-10-08 15:43:14+00:00,VOTE FOR @JohnFetterman .\nRepublican @DrOz w...,1578773016908951554,Richard79300111,0,0,1,0,1578773016908951554,,https://twitter.com/JohnFetterman/status/15787...,,,"[User(username='JohnFetterman', id=3622368202,...",0.0,1.0,1.0,1.0,VOTE FOR JohnFetterman .Republican DrOz will ...
2,2022-09-23 21:35:23+00:00,@the_vello Why does he always look like a SLOB...,1573425822970568709,sgtmolly06,0,0,6,0,1573419293273907211,,,1.57E+18,https://twitter.com/the_vello,"[User(username='the_vello', id=93330058, displ...",0.0,1.0,0.0,1.0,the_vello Why does he always look like a SLOB?...
3,2022-09-23 18:46:26+00:00,John Fetterman is a POS! Just a trust found ki...,1573383305566846976,aizenlorde,0,1,1,0,1573383305566846976,,,,,,0.0,1.0,1.0,1.0,John Fetterman is a POS! Just a trust found ki...
4,2022-09-23 16:04:47+00:00,"@DrOz I would love to see a sit-down, face-to-...",1573342622877626369,darthbanish_EP2,0,0,1,0,1573342057321619456,,,1.57E+18,https://twitter.com/DrOz,"[User(username='DrOz', id=38531995, displaynam...",0.0,1.0,1.0,1.0,"DrOz I would love to see a sit-down, face-to-f..."
5,2022-09-23 13:37:39+00:00,Baphomet is non binary and babies were slain f...,1573305596190904321,Nanaof313,0,0,3,0,1573305596190904321,,,,,,0.0,1.0,0.0,1.0,Baphomet is non binary and babies were slain f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,2022-10-08 05:47:14+00:00,"VIEW my latest #Rumble #RumbleVideo ""WHO IS JO...",1578623032465428481,TheReal_BigJohn,0,0,0,0,1578623032465428481,,,,,,2.0,0.0,0.0,1.0,"VIEW my latest Rumble RumbleVideo ""WHO IS JOHN..."
932,2022-10-07 15:44:50+00:00,While all the media blue checks are pounding #...,1578411032586113025,WilkowMajority,6,23,46,1,1578411032586113025,,,,,,2.0,0.0,0.0,1.0,While all the media blue checks are pounding H...
1031,2022-10-06 19:56:17+00:00,@CynthiaCoy8 @ph00ligan @MMMPeg59 @SeeHateGoAw...,1578111927112196096,PhillyComptonMW,3,1,7,0,1280501891336753155,,,1.57E+18,https://twitter.com/PhillyComptonMW,"[User(username='CynthiaCoy8', id=9706713411448...",0.0,1.0,1.0,1.0,avg8 criteria681 nwTGcpv...
1115,2022-10-02 17:12:56+00:00,"#JohnFetterman vote for me, I’m handicapped, j...",1576621264940171264,stryfe65,0,0,0,0,1576621264940171264,,,,,,0.0,1.0,1.0,1.0,"JohnFetterman vote for me, I’m handicapped, ju..."


# define our metric

In [9]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
def get_result(target,pred):
  '''returns accuracy, f1, precision, recall'''
  return accuracy_score(target,pred),f1_score(target,pred,average='weighted'),precision_score(target,pred),recall_score(target,pred)

# set seed

In [10]:
SEED = 0

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

# define model and get tokenizer

In [14]:
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [15]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")
def tokenize_roberta(data,max_len=128,tokenizer_roberta=tokenizer_roberta) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [16]:
def roberta_model(bert_model, max_len=128):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(128, activation='relu')(output)
    output = tf.keras.layers.Dense(1)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model

# Training using 5 folds

In [17]:
sum(sarcasm['Sarcasm'].values)

79.0

as we can see we only have 79 sarcastic tweets over 935! We should oversample them.

In [18]:
import random

def naive_oversample(x,y):
  if sum(y)/len(y)<0.5:
    minor=1.0
    diff = len(y)-sum(y)
  else:
    minor=0.0
    diff = abs(len(y)-sum(y)-sum(y))

  new_x=[]
  to_be_oversample=[]
  for i in range(len(x)):
    if int(y[i])==int(minor):
      to_be_oversample.append(x[i])

  y=list(y)
  x=list(x)
  for i in range(int(diff)):
    j = random.randint(0, len(to_be_oversample)-1)
    x.append(to_be_oversample[j])
    y.append(minor)

  return np.array(x),np.array(y)

    

In [19]:
import os

In [20]:
from sklearn.model_selection import KFold,StratifiedKFold

skf=KFold(n_splits=5, shuffle=True, random_state=SEED)
accuracy=[]
f1=[]
precision=[]
recall=[]
best_a=0
roberta = TFRobertaModel.from_pretrained('roberta-base')
best_model=roberta_model(roberta)
for train_index, test_index in skf.split(sarcasm['cleaned_tweets'].values,sarcasm['Sarcasm'].values):
  X_train, X_test = sarcasm['cleaned_tweets'].values[train_index], sarcasm['cleaned_tweets'].values[test_index]
  y_train, y_test = sarcasm['Sarcasm'].values[train_index], sarcasm['Sarcasm'].values[test_index]
  roberta = TFRobertaModel.from_pretrained('roberta-base')
  r_model = roberta_model(roberta)
  X_train, y_train = naive_oversample(X_train, y_train)
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  history = r_model.fit([train_input_ids,train_attention_masks], y_train, epochs=4, batch_size=32)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  y_pred=[]
  for i in result_roberta:
    if i<=0.5:
      y_pred.append(0)

    else:
      y_pred.append(1)
  a,b,c,d=get_result(y_test,y_pred)
  if a> best_a:
    best_model=r_model
    best_a=a
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

best_model.save_weights('/content/drive/MyDrive/model/sarcasm.h5')

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel f

Epoch 1/4
45/45 [==============================] - 49s 736ms/step - loss: 0.6645 - accuracy: 0.5003
Epoch 2/4
45/45 [==============================] - 34s 766ms/step - loss: 0.2953 - accuracy: 0.8829
Epoch 3/4
45/45 [==============================] - 36s 794ms/step - loss: 0.0785 - accuracy: 0.9728
Epoch 4/4
6/6 [==============================] - 5s 255ms/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the 

Epoch 1/4
45/45 [==============================] - 52s 794ms/step - loss: 0.6889 - accuracy: 0.4797
Epoch 2/4
45/45 [==============================] - 36s 797ms/step - loss: 0.5214 - accuracy: 0.6895
Epoch 3/4
45/45 [==============================] - 35s 787ms/step - loss: 0.1219 - accuracy: 0.9601
Epoch 4/4
6/6 [==============================] - 4s 254ms/step


...

Epoch 1/4
45/45 [==============================] - 52s 802ms/step - loss: 0.6734 - accuracy: 0.4930
Epoch 2/4
45/45 [==============================] - 36s 799ms/step - loss: 0.3143 - accuracy: 0.8456
Epoch 3/4
45/45 [==============================] - 36s 796ms/step - loss: 0.0619 - accuracy: 0.9826
Epoch 4/4
6/6 [==============================] - 4s 252ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
45/45 [==============================] - 52s 797ms/step - loss: 0.6855 - accuracy: 0.4783
Epoch 2/4
45/45 [==============================] - 36s 800ms/step - loss: 0.4693 - accuracy: 0.7294
Epoch 3/4
45/45 [==============================] - 36s 793ms/step - loss: 0.0882 - accuracy: 0.9790
Epoch 4/4
6/6 [==============================] - 4s 254ms/step


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/4
45/45 [==============================] - 52s 796ms/step - loss: 0.6946 - accuracy: 0.4759
Epoch 2/4
45/45 [==============================] - 36s 800ms/step - loss: 0.4833 - accuracy: 0.7037
Epoch 3/4
45/45 [==============================] - 36s 791ms/step - loss: 0.1519 - accuracy: 0.9434
Epoch 4/4
6/6 [==============================] - 4s 252ms/step


# Result

In [21]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.9016042780748663, f1: 0.8788396899347178, precision: 0.21333333333333332, recall: 0.12673992673992673
